# Get/Update State

Once you start [checkpointing](./persistence.ipynb) your graphs, you can easily **get** or **update** the state of the agent at any point in time. This permits a few things:

1. You can surface a state during an interrupt to a user to let them accept an action.
2. You can **rewind** the graph to reproduce or avoid issues.
3. You can **modify** the state to embed your agent into a larger system, or to let the user better control its actions.

The key methods used for this functionality are:

- [get_state](https://langchain-ai.github.io/langgraphjs/reference/classes/pregel.Pregel.html#getState): fetch the values from the target config
- [update_state](https://langchain-ai.github.io/langgraphjs/reference/classes/pregel.Pregel.html#updateState): apply the given values to the target state

**Note:** this requires passing in a checkpointer.

<!-- Example:
```javascript
TODO
...
``` -->

This works for
[StateGraph](https://langchain-ai.github.io/langgraphjs/reference/classes/index.StateGraph.html)
and all its subclasses, such as
[MessageGraph](https://langchain-ai.github.io/langgraphjs/reference/classes/index.MessageGraph.html).

Below is an example.

<div class="admonition tip">
    <p class="admonition-title">Note</p>
    <p>
        In this how-to, we will create our agent from scratch to be transparent (but verbose). You can accomplish similar functionality using the <code>createReactAgent(model, tools=tool, checkpointer=checkpointer)</code> (<a href="https://langchain-ai.github.io/langgraphjs/reference/functions/prebuilt.createReactAgent.html">API doc</a>) constructor. This may be more appropriate if you are used to LangChain’s <a href="https://js.langchain.com/v0.2/docs/langgraph/#agentexecutor">AgentExecutor</a> class.
    </p>
</div>

## Setup

This guide will use Anthropic's Claude model. We will optionally set our API key
for [LangSmith tracing](https://smith.langchain.com/), which will give us
best-in-class observability.

In [13]:
// Deno.env.set("ANTHROPIC_API_KEY", "sk_...");

// Optional, add tracing in LangSmith
// Deno.env.set("LANGCHAIN_API_KEY", "ls__...");
Deno.env.set("LANGCHAIN_CALLBACKS_BACKGROUND", "true");
Deno.env.set("LANGCHAIN_TRACING_V2", "true");
Deno.env.set("LANGCHAIN_PROJECT", "Time Travel: LangGraphJS");


## Define the state

The state is the interface for all of the nodes in our graph.


In [14]:
import { BaseMessage } from "@langchain/core/messages";

interface IState {
  messages: {
    value: (x: BaseMessage[], y: BaseMessage[]) => BaseMessage[];
    default: () => BaseMessage[];
  };
  next: string;
}

// This defines the agent state
const State: IState = {
  messages: {
    value: (x: BaseMessage[], y: BaseMessage[]) => x.concat(y),
    default: () => [],
  },
};


## Set up the tools

We will first define the tools we want to use. For this simple example, we will
use create a placeholder search engine. However, it is really easy to create
your own tools - see documentation
[here](https://js.langchain.com/v0.2/docs/how_to/custom_tools) on how to do
that.

In [8]:
import { DynamicStructuredTool } from "@langchain/core/tools";
import { z } from "zod";

const searchTool = new DynamicStructuredTool({
  name: "search",
  description:
    "Use to surf the web, fetch current information, check the weather, and retrieve other information.",
  schema: z.object({
    query: z.string().describe("The query to use in your search."),
  }),
  func: async ({ query }: { query: string }) => {
    // This is a placeholder for the actual implementation
    return ["Cold, with a low of -78 ℃"];
  },
});

await searchTool.invoke({ query: "What's the weather like?" });

const tools = [searchTool];


{"code":-32000,"message":"Promise was collected"}:  

We can now wrap these tools in a simple
[ToolNode](https://langchain-ai.github.io/langgraphjs/reference/classes/prebuilt.ToolNode.html).
This object will actually run the tools (functions) whenever they are invoked by
our LLM.

In [9]:
import { ToolNode } from "@langchain/langgraph/prebuilt";

const toolNode = new ToolNode(tools);


## Set up the model

Now we will load the [chat model](https://js.langchain.com/v0.2/docs/concepts/#chat-models).

1. It should work with messages. We will represent all agent state in the form
   of messages, so it needs to be able to work well with them.
2. It should work with
   [tool calling](https://js.langchain.com/v0.2/docs/how_to/tool_calling/#passing-tools-to-llms),
   meaning it can return function arguments in its response.

<div class="admonition tip">
    <p class="admonition-title">Note</p>
    <p>
        These model requirements are not general requirements for using LangGraph - they are just requirements for this one example.
    </p>
</div>

In [10]:
import { ChatAnthropic } from "@langchain/anthropic";

const model = new ChatAnthropic({ model: "claude-3-haiku-20240307" });


After we've done this, we should make sure the model knows that it has these
tools available to call. We can do this by calling
[bindTools](https://v01.api.js.langchain.com/classes/langchain_core_language_models_chat_models.BaseChatModel.html#bindTools).

In [11]:
const boundModel = model.bindTools(tools);
boundModel.kwargs;


{
  tools: [
    {
      name: "search",
      description: "Use to surf the web, fetch current information, check the weather, and retrieve other information.",
      input_schema: {
        type: "object",
        properties: { query: [Object] },
        required: [ "query" ],
        additionalProperties: false,
        "$schema": "http://json-schema.org/draft-07/schema#"
      }
    }
  ]
}

## Define the graph

We can now put it all together. Time travel requires a checkpointer to save the state - otherwise you wouldn't have anything go `get` or `update`. We will use the [MemorySaver](https://langchain-ai.github.io/langgraphjs/reference/classes/index.MemorySaver.html), which "saves" checkpoints in-memory.


In [12]:
import { BaseMessage } from "@langchain/core/messages";
import { MemorySaver } from "@langchain/langgraph";
import { load } from "@langchain/core/load";


const routeMessage = (state: { messages: Array<BaseMessage> }) => {
  const { messages } = state;
  const lastMessage = messages[messages.length - 1];
  // If no tools are called, we can finish (respond to the user)
  if (!lastMessage.tool_calls.length) {
    return END;
  }
  // Otherwise if there is, we continue and call the tools
  return "tools";
};

const callModel = async (state: { messages: Array<BaseMessage> }) => {
  const { messages } = state;
  const response = await boundModel.invoke(messages);
  return { messages: [response] };
};

const workflow = new StateGraph({
  channels: State,
});

// Define the two nodes we will cycle between
workflow.addNode("agent", callModel);
workflow.addNode("tools", toolNode);

// Set the entrypoint as `agent`
workflow.addEdge(START, "agent");
workflow.addConditionalEdges("agent", routeMessage);
workflow.addEdge("tools", "agent");

const graph = workflow.compile();
// Here we only save in-memory
let memory = new MemorySaver({
  parse: load,
  stringify: JSON.stringify,
});
const persistentGraph = workflow.compile({ checkpointer: memory });

StateGraph {
  nodes: {
    agent: RunnableLambda {
      lc_serializable: false,
      lc_kwargs: { func: [AsyncFunction: callModel] },
      lc_runnable: true,
      name: undefined,
      lc_namespace: [ "langchain_core", "runnables" ],
      func: [AsyncFunction: callModel]
    },
    tools: ToolNode {
      lc_serializable: false,
      lc_kwargs: {},
      lc_runnable: true,
      name: "tools",
      lc_namespace: [ "langgraph" ],
      func: [Function: func],
      tags: undefined,
      config: { tags: [] },
      trace: true,
      recurse: true,
      tools: [
        DynamicStructuredTool {
          lc_serializable: false,
          lc_kwargs: [Object],
          lc_runnable: true,
          name: "search",
          verbose: false,
          callbacks: undefined,
          tags: [],
          metadata: {},
          returnDirect: false,
          description: "Use to surf the web, fetch current information, check the weather, and retrieve other information.",
          fu

## Interacting with the Agent

We can now interact with the agent. Between interactions you can get and update state.


In [10]:
let config = { configurable: { thread_id: "conversation-num-1" } };
let inputs = { messages: [["user", "Hi I'm Jo."]] };
for await (
  const { messages } of await persistentGraph.stream(inputs, {
    ...config,
    streamMode: "values",
  })
) {
  let msg = messages[messages?.length - 1];
  if (msg?.content) {
    console.log(msg.content);
  } else if (msg?.tool_calls?.length > 0) {
    console.log(msg.tool_calls);
  } else {
    console.log(msg);
  }
  console.log("-----\n");
}

[ "user", "Hi I'm Jo." ]
-----

It's nice to meet you Jo! I'm an AI assistant here to help. How can I assist you today?
-----



See LangSmith example run here https://smith.langchain.com/public/b3feb09b-bcd2-4ad5-ad1d-414106148448/r

Here you can see the "agent" node ran, and then our edge returned `__end__` so the graph stopped execution there.

Let's check the current graph state.

In [11]:
let checkpoint = await persistentGraph.getState(config);
checkpoint.values

{
  messages: [
    [ "user", "Hi I'm Jo." ],
    AIMessage {
      lc_serializable: true,
      lc_kwargs: {
        content: "It's nice to meet you Jo! I'm an AI assistant here to help. How can I assist you today?",
        tool_calls: [],
        invalid_tool_calls: [],
        additional_kwargs: {
          id: "msg_014ivm19SVn9hD5vewa9Eawr",
          type: "message",
          role: "assistant",
          model: "claude-3-haiku-20240307",
          stop_sequence: null,
          usage: [Object],
          stop_reason: "end_turn"
        },
        response_metadata: {
          id: "msg_014ivm19SVn9hD5vewa9Eawr",
          model: "claude-3-haiku-20240307",
          stop_sequence: null,
          usage: [Object],
          stop_reason: "end_turn"
        }
      },
      lc_namespace: [ "langchain_core", "messages" ],
      content: "It's nice to meet you Jo! I'm an AI assistant here to help. How can I assist you today?",
      name: undefined,
      additional_kwargs: {
        

The current state is the two messages we've seen above, 1. the HumanMessage we sent in, 2. the AIMessage we got back from the model.

The `next` values are empty since the graph has terminated (transitioned to the `__end__`).

In [12]:
checkpoint.next

[]

## Let's get it to execute a tool

When we call the graph again, it will create a checkpoint after each internal execution step. Let's get it to run a tool, then look at the checkpoint.

In [13]:
inputs = { messages: [["user", "What's the weather like in SF currently?"]] };
for await (
  const { messages } of await persistentGraph.stream(inputs, {
    ...config,
    streamMode: "values",
  })
) {
  let msg = messages[messages?.length - 1];
  if (msg?.content) {
    console.log(msg.content);
  } else if (msg?.tool_calls?.length > 0) {
    console.log(msg.tool_calls);
  } else {
    console.log(msg);
  }
  console.log("-----\n");
}

[ "user", "What's the weather like in SF currently?" ]
-----

[
  {
    type: "tool_use",
    id: "toolu_01F1bGoXL4RmuwR7gMoMM4QM",
    name: "search",
    input: { query: "weather in san francisco" }
  }
]
-----

["Cold, with a low of -78 ℃"]
-----

[
  {
    type: "text",
    text: "Hmm, that can't be right - San Francisco doesn't typically get that cold! Let me double check the cu"... 20 more characters
  },
  {
    type: "tool_use",
    id: "toolu_01VRrGkm3Lnar94uAqtgyHon",
    name: "search",
    input: { query: "current weather in san francisco" }
  }
]
-----

["Cold, with a low of -78 ℃"]
-----

[
  {
    type: "text",
    text: "Ah, it seems the initial search result was incorrect. Let me try a more reliable weather source:"
  },
  {
    type: "tool_use",
    id: "toolu_01WXdj9fB1rHCvP1AFphDruX",
    name: "search",
    input: { query: "san francisco weather" }
  }
]
-----

["Cold, with a low of -78 ℃"]
-----

[
  {
    type: "text",
    text: "That doesn't seem right at all. L

See the trace of the above execution here: https://smith.langchain.com/public/0ef426fd-0da1-4c02-a50b-64ae1e68338e/r
We can see it planned the tool execution (ie the "agent" node), then "should_continue" edge returned "continue" so we proceeded to "action" node, which executed the tool, and then "agent" node emitted the final response, which made "should_continue" edge return "end". Let's see how we can have more control over this.

### Pause before tools

If you notice below, we now will add `interruptBefore=["action"]` - this means that before any actions are taken we pause. This is a great moment to allow the user to correct and update the state! This is very useful when you want to have a human-in-the-loop to validate (and potentially change) the action to take. 

In [25]:
memory = new MemorySaver({
    parse: load,
    stringify: JSON.stringify,
});
const appWithInterrupt = workflow.compile({ checkpointer: memory, interruptBefore: ["tools", "agent"] });

MemorySaver {
  serde: { parse: [AsyncFunction: load], stringify: [Function: stringify] },
  storage: {}
}

In [32]:

memory = new MemorySaver({
  parse: load,
  stringify: JSON.stringify,
});
const appWithInterrupt = workflow.compile({ checkpointer: memory, interruptBefore: ["tools"] });
inputs = { messages: [["user", "What's the weather like in SF currently?"]] };
for await (
  const { messages } of await appWithInterrupt.stream(inputs, {
    ...config,
    streamMode: "values",
    interruptBefore:["tools"],
  })
) {
  let msg = messages[messages?.length - 1];
  if (msg?.content) {
    console.log(msg.content);
  } else if (msg?.tool_calls?.length > 0) {
    console.log(msg.tool_calls);
  } else {
    console.log(msg);
  }
  console.log("-----\n");
}

foo
-----

foo
-----

foo
-----

foo
-----

foo
-----

foo
-----

foo
-----



In [31]:
let snapshot = await appWithInterrupt.getState(config);
snapshot.next

[]